<a href="https://colab.research.google.com/github/GianmarcoLattaruolo/Vision_Project/blob/main/Vision_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First of all let'smount our vision drive.

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir(r'/content/drive/MyDrive/GeoEstimation')
print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/GeoEstimation


Now we need to install some specific packages with certain version to account for the original environment in which the paper results were obtained. We need to match these conditions:
```python
  - python=3.8
  - msgpack-python=1.0.0
  - pandas=1.1.5
  - yaml=0.2.5
  - tqdm=4.50
  - cudatoolkit=10.2
  - pytorch=1.6
  - torchvision=0.7
  - pytorch-lightning=1.0.1
  - pip
  - pip:
    - s2sphere==0.2.5
```

In [3]:
!python --version
!pip install pandas==1.1.5
!pip install msgpack==1.0.0
!pip install tqdm==4.50
!pip install torch==1.6
!pip install torchvision==0.7
!pip install torchtext==0.8.0 torch==1.6 pytorch-lightning==1.0.1 
!pip install s2sphere==0.2.5

#we lack only this to packages
#!pip install yaml==0.2.5
#!pip install cudatoolkit==10.2

Python 3.8.10
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 94.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2022.12.0 requires pandas>=1.3, but you have pandas 1.1.5 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.5/303.5 KB 2.3 MB/s eta 0:00:00
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.4
    Uninstalling msgpack-1.0.4:
      Successfully uninstalled msgpack-1.0.4
Looking in indexes: https://pypi.org/simple, https://us-p

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.8/748.8 MB 2.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.6.0 which is incompatible.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 1.6.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.6.0 which is incompatible.
fastai 2.7.10 requires torch<1.14,>=1.7, but you have torch 1.6.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━

In [18]:
import msgpack
#print("msgpack:", msgpack.__version__) #msgpack sembra non avere l'attributo __version__ nella versiona in cui lo abbiamo installato
import pandas
print("pandas:",pandas.__version__)
import yaml
print("yaml:",yaml.__version__)
import tqdm
print("tqdm:",tqdm.__version__)
#import cudatoolkit
#print("cudatoolkit:",cudatoolkit.__version__)
import torch
print("pytorch:",torch.__version__)
import torchvision
print("torchvision:",torchvision.__version__)
import pytorch_lightning as pl
print("pytorch_lightning:",pl.__version__)

pandas: 1.1.5
yaml: 6.0
tqdm: 4.64.1
pytorch: 1.13.1+cu116
torchvision: 0.7.0
pytorch_lightning: 1.0.1


# Reproduce paper results


We may need to run this cells once because thereafter all will be saved on our vision drive.

In [ ]:
!git clone https://github.com/TIBHannover/GeoEstimation.git && cd GeoEstimation
os.chdir(r'/content/drive/MyDrive/GeoEstimation')
print(os.getcwd())
!mkdir -p resources/s2_cells
!wget https://raw.githubusercontent.com/TIBHannover/GeoEstimation/original_tf/geo-cells/cells_50_5000.csv -O resources/s2_cells/cells_50_5000.csv
!wget https://raw.githubusercontent.com/TIBHannover/GeoEstimation/original_tf/geo-cells/cells_50_2000.csv -O resources/s2_cells/cells_50_2000.csv
!wget https://raw.githubusercontent.com/TIBHannover/GeoEstimation/original_tf/geo-cells/cells_50_1000.csv -O resources/s2_cells/cells_50_1000.csv
!mkdir -p models/base_M
!wget https://github.com/TIBHannover/GeoEstimation/releases/download/pytorch/epoch.014-val_loss.18.4833.ckpt -O models/base_M/epoch=014-val_loss=18.4833.ckpt
!wget https://github.com/TIBHannover/GeoEstimation/releases/download/pytorch/hparams.yaml -O models/base_M/hparams.yaml
# download im2gps testset
!mkdir -p resources/images/im2gps
!wget http://graphics.cs.cmu.edu/projects/im2gps/gps_query_imgs.zip -O resources/images/im2gps.zip
!unzip resources/images/im2gps.zip -d resources/images/im2gps/
!wget https://raw.githubusercontent.com/TIBHannover/GeoEstimation/original_tf/meta/im2gps_places365.csv -O resources/images/im2gps_places365.csv
!wget https://raw.githubusercontent.com/TIBHannover/GeoEstimation/original_tf/meta/im2gps3k_places365.csv -O resources/images/im2gps3k_places365.csv


Cloning into 'GeoEstimation'...
remote: Enumerating objects: 456, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 456 (delta 11), reused 7 (delta 7), pack-reused 436
Receiving objects: 100% (456/456), 1.88 MiB | 4.96 MiB/s, done.
Resolving deltas: 100% (245/245), done.
/content/drive/MyDrive/GeoEstimation
--2023-02-02 15:57:37--  https://raw.githubusercontent.com/TIBHannover/GeoEstimation/original_tf/geo-cells/cells_50_5000.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177214 (173K) [text/plain]
Saving to: ‘resources/s2_cells/cells_50_5000.csv’

resources/s2_cells/ 100%[===================>] 173.06K  --.-KB/s    in 0.02s   

2023-02-02 15:57:37 (10.1 MB/s) - ‘resources/s2_cells/cells_50

This two commands give the same error:
```python
OSError: /usr/local/lib/python3.8/dist-packages/torchtext/_torchtext.so: undefined symbol: _ZN3c104impl23ExcludeDispatchKeyGuardC1ENS_14DispatchKeySetE
```

In [43]:
#Inference with pre-trained model:
!python3 -m classification.inference --image_dir resources/images/im2gps/
print("\n\n\n")
#Test on Already Trained Model
!python -m classification.test

Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/content/drive/MyDrive/GeoEstimation/classification/inference.py", line 8, in <module>
    from classification.train_base import MultiPartitioningClassifier
  File "/content/drive/MyDrive/GeoEstimation/classification/train_base.py", line 10, in <module>
    import pytorch_lightning as pl
  File "/usr/local/lib/python3.8/dist-packages/pytorch_lightning/__init__.py", line 56, in <module>
    from pytorch_lightning.core import LightningDataModule, LightningModule
  File "/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/__init__.py", line 14, in <module>
    from pytorch_lightning.core.datamodule import LightningDataModule
  File "/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/datamodule.py", line 22, in <module>
    

These are the other commands for Training from Scratch (that we won't use maybe):

```python
# download and preprocess images
wget https://github.com/TIBHannover/GeoEstimation/releases/download/v1.0/mp16_urls.csv -O resources/mp16_urls.csv
wget https://github.com/TIBHannover/GeoEstimation/releases/download/pytorch/yfcc25600_urls.csv -O resources/yfcc25600_urls.csv 
python download_images.py --output resources/images/mp16 --url_csv resources/mp16_urls.csv --shuffle
python download_images.py --output resources/images/yfcc25600 --url_csv resources/yfcc25600_urls.csv --shuffle --size_suffix ""

# assign cell(s) for each image using the original meta information
wget https://github.com/TIBHannover/GeoEstimation/releases/download/v1.0/mp16_places365.csv -O resources/mp16_places365.csv
wget https://github.com/TIBHannover/GeoEstimation/releases/download/pytorch/yfcc25600_places365.csv -O resources/yfcc25600_places365.csv
python partitioning/assign_classes.py
# remove images that were not downloaded 
python filter_by_downloaded_images.py

# train geo model from scratch
python -m classification.train_base --config config/baseM.yml
```

# Vision and Cognition Systems - Project


In [25]:
os.chdir(r'/content/drive/MyDrive/GeoEstimation/resources/images/im2gps')
print(len(os.listdir()))
os.chdir(r'/content/drive/MyDrive/GeoEstimation')
print(os.getcwd())
import torch
print(torch.cuda.is_available())

# Output would be True if Pytorch is using GPU otherwise it would be False.
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

237
/content/drive/MyDrive/GeoEstimation
True
1
Tesla T4


In [ ]:
#@title
#libraries to import
#known
import pandas as pd
import numpy as np
import os
import re
import torchvision
import torch
import PIL
from PIL import Image
from PIL import ImageFile
import sys
import time
from math import ceil



#Unknown
from typing import Union
from io import BytesIO
import random
from argparse import Namespace, ArgumentParser
from pathlib import Path
from multiprocessing import Pool
from functools import partial
import requests
import logging
import json
import yaml
from tqdm.auto import tqdm
#from classification.train_base import MultiPartitioningClassifier
#from classification.dataset import FiveCropImageDataset

#to divide
from classification import utils_global
from classification.s2_utils import Partitioning, Hierarchy
from classification.dataset import MsgPackIterableDatasetMultiTargetWithDynLabels


The main link and paper that we need to follow is [this](https://github.com/TIBHannover/GeoEstimation) and [this](https://github.com/TIBHannover/GeoEstimation/releases/) for the pretrained models.

Davide ha trovato questo che forse è meglio [kaggle](https://www.kaggle.com/code/habedi/inspect-the-dataset/data)

[Qui](https://qualinet.github.io/databases/image/world_wide_scale_geotagged_image_dataset_for_automatic_image_annotation_and_reverse_geotagging/) ci sono dei links che potrebbero essere usati con colab col comando !wget.